In [ ]:
# Get necessary imports
import pandas as pd
import openpyxl
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment, numbers
from openpyxl.utils import get_column_letter
from datetime import datetime
import calendar
from copy import copy  # For copying Excel cell styles

In [ ]:
# Define target Excel file 
output_path = 'weekly.xlsx'

# Define the sheets and paths to process with constants for easy reference
# Format: (file_path, sheet_name, display_name)
sheet_info = [
    ('test2/may25-final.xlsx', 'Sheet1', 'May 25'),   # Last month raw sheet
    ('test2/June24_Invoice.xlsx', 'Raw data June 24', 'June 24'),        # Last year raw sheet
    ('test2/June25.xlsx', 'Sheet1', 'June 25')                # Latest month raw sheet
]

latest_path, latest_sheet, latest_display = sheet_info[-1]
latest_df = pd.read_excel(latest_path, sheet_name=latest_sheet)
max_invoice_day = pd.to_datetime(latest_df['InvoiceDate'], dayfirst=True, errors='coerce').dt.day.max()




# Define sessions_info
sessions_info = [
    ('test2/May_2025_Daily traffic (2).xlsx', 'download - 2025-01-08T160122.10', 'May 25'), # Last month session
    ('test2/June Traffic -2024.xlsx', 'download - 2025-06-03T09524 (2)', 'June 24'),  # Last year session
    ('test2/June_2025_Daily traffic.xlsx', 'download - 2025-01-08T160122.10', 'June 25')   # Current month session
]

# Create unique identifiers by combining file path and sheet name
LAST_MONTH_ID = f"{sheet_info[0][0]}_{sheet_info[0][1]}"
LAST_YEAR_ID = f"{sheet_info[1][0]}_{sheet_info[1][1]}"
CURRENT_ID = f"{sheet_info[2][0]}_{sheet_info[2][1]}"

# Dynamic constants extracted from sheet_info for easy reference
LAST_MONTH_PATH = sheet_info[0][0]      # File path for last month
LAST_MONTH_SHEET = sheet_info[0][1]     # Sheet name for last month
LAST_MONTH_DISPLAY = sheet_info[0][2]   # Display name for last month

LAST_YEAR_PATH = sheet_info[1][0]       # File path for last year
LAST_YEAR_SHEET = sheet_info[1][1]      # Sheet name for last year
LAST_YEAR_DISPLAY = sheet_info[1][2]    # Display name for last year

CURRENT_PATH = sheet_info[2][0]         # File path for current month
CURRENT_SHEET = sheet_info[2][1]        # Sheet name for current month
CURRENT_DISPLAY = sheet_info[2][2]      # Display name for current month

# Create dynamic periods list from sheet_info display names
periods = [LAST_MONTH_DISPLAY, LAST_YEAR_DISPLAY, CURRENT_DISPLAY]

print("📊 Dynamic Configuration Loaded:")
print("=" * 40)
print(f"Last Month:   {LAST_MONTH_DISPLAY} (ID: {LAST_MONTH_ID})")
print(f"Last Year:    {LAST_YEAR_DISPLAY} (ID: {LAST_YEAR_ID})")  
print(f"Current:      {CURRENT_DISPLAY} (ID: {CURRENT_ID})")
print(f"Periods:      {periods}")
print("=" * 40)

# Target sheet information
TARGET_PATH = 'test2/Target_June_25.xlsx'
TARGET_SHEET = 'Target-June25'

In [ ]:
# Get target sums by day and channel using constants - WITH DEBUG OUTPUT
print("🔍 DEBUGGING TARGET DATA STRUCTURE")
print("=" * 50)

target_df = pd.read_excel(TARGET_PATH, sheet_name=TARGET_SHEET)
print(f"📊 Raw Target Data Shape: {target_df.shape}")
print(f"📋 Target Data Columns: {list(target_df.columns)}")
print("\n📅 First 10 rows of Target Data:")
display(target_df.head(10))

print("\n🏷️ Unique Channels in Target Data:")
print(target_df['Channel'].unique())

print("\n🏷️ Unique Categories in Target Data:")
if 'Category' in target_df.columns:
    print(target_df['Category'].unique())
else:
    print("No 'Category' column found in target data")

# Modified to include 'Category' in the grouping if it exists
if 'Category' in target_df.columns:
    target_sums = target_df.groupby(['Date', 'Channel', 'Category'])['Target'].sum().unstack(level=['Channel', 'Category'], fill_value=0).round(6)
else:
    target_sums = target_df.groupby(['Date', 'Channel'])['Target'].sum().unstack(level='Channel', fill_value=0).round(6)

print(f"\n📊 Processed Target Sums Shape: {target_sums.shape}")
print("\n📋 Target Sums Sample:")
display(target_sums.head())

print("=" * 50)

In [ ]:
# 📅 Week Analysis Configuration - User Input Based
from datetime import datetime
import calendar

print("🗓️  WEEK ANALYSIS CONFIGURATION")
print("=" * 50)
print("📝 Choose your preferred starting day for weekly analysis:")
print("   Options: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday")
print()

# Get the first day of the month from user
while True:
    first_day = input('Enter the first day of the month (Monday/Tuesday/Wednesday/Thursday/Friday/Saturday/Sunday): ').strip().lower()
    if first_day in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']:
        break
    print('Invalid input! Please enter a valid day name.')

# Create a mapping of days to their position in a week (0=Monday to 6=Sunday)
day_positions = {
    'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
    'friday': 4, 'saturday': 5, 'sunday': 6
}
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Get position of the first day (0-6, where 0 is Monday)
first_day_position = day_positions[first_day]
start_day = day_names[first_day_position].capitalize()

print(f"✅ First day of month set to: {start_day}")
print(f"📊 Configuration: Week analysis starts based on {start_day} as first day")
print("=" * 50)

def get_week_info(day_of_month, first_day_pos):
    """Get week information for a given day.
    Returns (week_number, is_first_partial_week)"""
    # For days in the first partial week
    if first_day_pos > 0:  # If month doesn't start on Monday
        days_till_next_monday = 7 - first_day_pos
        if day_of_month <= days_till_next_monday:
            return 1, True
        # Adjust day number to calculate remaining weeks
        adjusted_day = day_of_month - days_till_next_monday
        return (adjusted_day - 1) // 7 + 2, False
    else:  # If month starts on Monday
        return (day_of_month - 1) // 7 + 1, False

def get_week_number(day_of_month, first_day_pos):
    """Get week number for a given day of month"""
    week_num, _ = get_week_info(day_of_month, first_day_pos)
    return week_num

def get_week_label(week_num):
    """Get descriptive week label"""
    return f"Week {week_num}"

def get_day_name(day_number, first_day_pos):
    """Get the day name for a given day of month"""
    # Calculate the day of week (0-6, where 0 is Monday)
    day_of_week = (first_day_pos + day_number - 1) % 7
    return day_names[day_of_week]

# Test the calculation
print(f"\n📊 Week Calculation Test (First day: {start_day})")
print("=" * 50)
for day in range(1, 16):  # Show first 15 days
    week_num = get_week_number(day, first_day_position)
    week_label = get_week_label(week_num)
    day_name = get_day_name(day, first_day_position)
    print(f"Day {day:2d} ({day_name:9s}) → {week_label}")

# Calculate total weeks for a typical 31-day month
max_weeks = max([get_week_number(day, first_day_position) for day in range(1, 32)])
print(f"\n📈 Total weeks in a 31-day month: {max_weeks}")
print(f"📋 Analysis will group days based on {start_day} as first day of month")
print("=" * 50)

# Store variables for use in other cells
start_day_num = first_day_position  # For compatibility with existing code


In [ ]:
# Collect week-wise and TYPE-wise sums for each sheet using simplified week calculation
results = []
type_results = []
idg_results = []  # New list for IDG results

for path, sheet, display_name in sheet_info:
    df = pd.read_excel(path, sheet_name=sheet)
    filtered_df = df[~df['idg'].isin(['FOC', 'Remove', 'WRT'])].copy()
    filtered_df['InvoiceDay'] = pd.to_datetime(filtered_df['InvoiceDate'], dayfirst=True, errors='coerce').dt.day


    original_len = len(filtered_df)
    filtered_df = filtered_df[filtered_df['InvoiceDay'] <= max_invoice_day]
    print(f"📉 {display_name}: Filtered {original_len - len(filtered_df)} rows with InvoiceDay > {max_invoice_day}")


    
    # Add week number calculation using the simplified function
    filtered_df['WeekNumber'] = filtered_df['InvoiceDay'].apply(
        lambda day: get_week_number(day, first_day_position)
    )
    
    # Map CC to Jumbo.ae in the TYPE column
    filtered_df['TYPE'] = filtered_df['TYPE'].replace('CC', 'Jumbo.ae')
    
    # Week-wise sum - use a unique identifier combining file path and sheet name
    unique_id = f"{path}_{sheet}"
    invoice_week_sum = filtered_df.groupby('WeekNumber')['Amount Invoiced W.O. VAT'].sum()
    results.append((unique_id, invoice_week_sum, display_name))
    
    # TYPE-wise sum for Jumbo.ae and EA by week - use unique identifier
    filtered_type = filtered_df[filtered_df['TYPE'].isin(['Jumbo.ae', 'EA'])]
    sum_by_week_type = filtered_type.groupby(['WeekNumber', 'TYPE'])['Amount Invoiced W.O. VAT'].sum().unstack(fill_value=0)
    type_results.append((unique_id, sum_by_week_type, display_name))
    
    # IDG-wise sum by week (new addition) - use unique identifier
    sum_by_week_idg = filtered_df.pivot_table(
        values='Amount Invoiced W.O. VAT',
        index='WeekNumber',
        columns='idg',
        aggfunc='sum',
        fill_value=0
    )
    idg_results.append((unique_id, display_name, sum_by_week_idg))

type_results

# Target Data Processing Functions with Channel Filtering and Debug Output
print("🎯 TARGET DATA PROCESSING WITH CHANNEL FILTERING")
print("=" * 60)

def process_target_data_by_week_and_channel(channel_filter=None):
    """
    Process target data and group by week and IDG (Category), with optional channel filtering
    
    Args:
        channel_filter: str, optional - Filter by 'Jumbo.ae' or 'EA' or None for all
    
    Returns:
        DataFrame with weeks as index and IDG (Category) as columns
    """
    print(f"\n🔄 Processing target data for channel: {channel_filter or 'ALL CHANNELS'}")
    
    # Read target data
    target_df = pd.read_excel(TARGET_PATH, sheet_name=TARGET_SHEET)
    
    print(f"📊 Original target data shape: {target_df.shape}")
    
    # Apply channel filter if specified
    if channel_filter:
        filtered_target = target_df[target_df['Channel'] == channel_filter].copy()
        print(f"🔍 After {channel_filter} filter: {filtered_target.shape}")
        
        if filtered_target.empty:
            print(f"⚠️ WARNING: No target data found for channel '{channel_filter}'")
            print(f"📅 Available channels: {target_df['Channel'].unique()}")
            return pd.DataFrame()
    else:
        filtered_target = target_df.copy()
    
    # Use the 'Date' column directly as 'Day' as it contains day numbers
    filtered_target['Day'] = filtered_target['Date']
    
    # Ensure 'Day' column is numeric and handle potential errors (e.g., non-numeric values)
    filtered_target['Day'] = pd.to_numeric(filtered_target['Day'], errors='coerce')

    # Remove rows with invalid dates (where 'Day' could not be converted to numeric)
    before_date_filter = len(filtered_target)
    filtered_target = filtered_target.dropna(subset=['Day'])
    after_date_filter = len(filtered_target)
    
    if before_date_filter != after_date_filter:
        print(f"⚠️ Removed {before_date_filter - after_date_filter} rows with invalid 'Day' values (non-numeric or empty).")
    
    # Convert 'Day' to integer type after cleaning
    if not filtered_target.empty:
        filtered_target['Day'] = filtered_target['Day'].astype(int)

    original_len = len(filtered_target)
    filtered_target = filtered_target[filtered_target['Day'] <= max_invoice_day]
    print(f"📉 {display_name}: Filtered {original_len - len(filtered_target)} rows with Day > {max_invoice_day}")
    # Add week number calculation
    filtered_target['WeekNumber'] = filtered_target['Day'].apply(
        lambda day: get_week_number(day, first_day_position)
    )
    
    print(f"📅 Date range in target: {filtered_target['Day'].min()} to {filtered_target['Day'].max()}")
    print(f"📋 Week range: {filtered_target['WeekNumber'].min()} to {filtered_target['WeekNumber'].max()}")
    
    # Check if Category column exists and use it, otherwise use Channel as IDG
    if 'Category' in filtered_target.columns:
        print(f"🏷️ Using 'Category' column for IDG grouping")
        print(f"📅 Available categories: {filtered_target['Category'].unique()}")
        
        # Group by Week and Category to get target by week and IDG
        target_by_week = filtered_target.groupby(['WeekNumber', 'Category'])['Target'].sum().unstack(level='Category', fill_value=0)
    else:
        print(f"🏷️ No 'Category' column found, using 'Channel' for IDG grouping")
        
        # Group by Week and Channel to get target by week
        target_by_week = filtered_target.groupby(['WeekNumber', 'Channel'])['Target'].sum().unstack(level='Channel', fill_value=0)
    
    # Clean column names
    target_by_week.columns.name = None
    
    print(f"📊 Final target pivot shape: {target_by_week.shape}")
    print(f"📋 Target IDGs/Categories: {list(target_by_week.columns)}")
    
    return target_by_week

# Test target processing for all channels
print("\n🔬 TESTING TARGET DATA PROCESSING:")
print("-" * 40)

# Test 1: All channels
all_channels_target = process_target_data_by_week_and_channel()
if not all_channels_target.empty:
    print("\n📋 ALL CHANNELS TARGET DATA:")
    display(all_channels_target.round(2))

# Test 2: Jumbo.ae only  
jumbo_target = process_target_data_by_week_and_channel('Jumbo.ae')
if not jumbo_target.empty:
    print("\n📋 JUMBO.AE TARGET DATA:")
    display(jumbo_target.round(2))

# Test 3: EA only
ea_target = process_target_data_by_week_and_channel('EA')
if not ea_target.empty:
    print("\n📋 EA TARGET DATA:")
    display(ea_target.round(2))

print("\n=" * 60)

In [ ]:
# Get the first sheet info
first_path, first_sheet, first_display = sheet_info[0]

# Read and process the data
df = pd.read_excel(first_path, sheet_name=first_sheet)

# Filter out unwanted IDG values and create pivot table
filtered_df = df[~df['idg'].isin(['FOC', 'Remove', 'WRT'])].copy()
filtered_df['InvoiceDay'] = pd.to_datetime(filtered_df['InvoiceDate'], dayfirst=True, errors='coerce').dt.day

# Create comprehensive IDG pivot table function for reusability

def process_sheet_data_by_type(path, sheet_name, display_name, type_filter=None):
    """Process a single sheet and return IDG pivot by week, optionally filtered by TYPE"""
    df = pd.read_excel(path, sheet_name=sheet_name)
    filtered_df = df[~df['idg'].isin(['FOC', 'Remove', 'WRT'])].copy()
    
    # Apply TYPE filter if specified
    if type_filter:
        # Map CC to Jumbo.ae first
        filtered_df['TYPE'] = filtered_df['TYPE'].replace('CC', 'Jumbo.ae')
        filtered_df = filtered_df[filtered_df['TYPE'] == type_filter]
    
    filtered_df['InvoiceDay'] = pd.to_datetime(filtered_df['InvoiceDate'], dayfirst=True, errors='coerce').dt.day

    original_len = len(filtered_df)
    filtered_df = filtered_df[filtered_df['InvoiceDay'] <= max_invoice_day]
    print(f"📉 {display_name}: Filtered {original_len - len(filtered_df)} rows with InvoiceDay > {max_invoice_day}")
    
    # Add week number calculation using simplified function
    filtered_df['WeekNumber'] = filtered_df['InvoiceDay'].apply(
        lambda day: get_week_number(day, first_day_position)
    )
    
    # Create pivot table with IDG on rows and weeks on columns
    idg_pivot = filtered_df.pivot_table(
        values='Amount Invoiced W.O. VAT',
        index='idg',
        columns='WeekNumber',
        aggfunc='sum',
        fill_value=0
    )
    
    return idg_pivot

def create_comprehensive_pivot_table(sheet_info, periods, type_filter=None, table_name="IDG"):
    """Create a comprehensive pivot table with weeks as super columns and periods as sub-columns, including target data"""
    
    print(f"\n🔄 Creating {table_name} Pivot Table with Target Data")
    print(f"🏷️ Type Filter: {type_filter or 'None (All Types)'}")
    
    # Process target data based on type filter
    # Map type_filter to appropriate channel for target data
    target_channel_filter = None
    if type_filter == "EA":
        target_channel_filter = "EA"
    elif type_filter == "Jumbo.ae":
        target_channel_filter = "Jumbo.ae"
    # For Overall analysis (type_filter=None), we use all channels
    
    print(f"🎯 Processing target data with channel filter: {target_channel_filter or 'ALL CHANNELS'}")
    target_by_week = process_target_data_by_week_and_channel(target_channel_filter)
    
    # Process all sheets with optional TYPE filter
    # Using display_name as the key in the dictionary instead of sheet_name
    sheet_data = {}
    for path, sheet_name, display_name in sheet_info:
        print(f"📄 Processing {display_name} data...")
        sheet_data[display_name] = process_sheet_data_by_type(path, sheet_name, display_name, type_filter)
    
    # Get all unique IDG values across all sheets and target data
    all_idgs = set()
    for data in sheet_data.values():
        all_idgs.update(data.index)
    
    # Add IDGs from target data if available
    if not target_by_week.empty:
        all_idgs.update(target_by_week.columns)
        print(f"🎯 Target IDGs found: {list(target_by_week.columns)}")
    else:
        print(f"⚠️ No target data available for filter: {target_channel_filter}")
    
    all_idgs = sorted(list(all_idgs))
    print(f"📋 Combined IDGs: {all_idgs}")
    
    # Calculate maximum week number across all data
    max_week = 1
    for data in sheet_data.values():
        if len(data.columns) > 0:
            max_week = max(max_week, max(data.columns))
    
    if not target_by_week.empty and len(target_by_week.index) > 0:
        max_week = max(max_week, max(target_by_week.index))
    
    print(f"📅 Maximum week number: {max_week}")
    
    # Create the comprehensive pivot table
    weeks = list(range(1, max_week + 1))
    
    # Create multi-level column index with target, comparison columns and totals
    # Order: Last Month, Last Year, Target, Current Month, vs Target %, vs Last Year %, vs Last Month %
    column_tuples = []
    for week in weeks:
        week_label = get_week_label(week)
        # Add period columns in specific order
        column_tuples.append((week_label, LAST_MONTH_DISPLAY))
        column_tuples.append((week_label, LAST_YEAR_DISPLAY))
        column_tuples.append((week_label, 'Target'))
        column_tuples.append((week_label, CURRENT_DISPLAY))
        # Add comparison columns
        column_tuples.append((week_label, 'v/s Target %'))
        column_tuples.append((week_label, 'v/s Last Year %'))
        column_tuples.append((week_label, 'v/s Last Month %'))
    # Add Total columns in same order
    column_tuples.append(('Total', LAST_MONTH_DISPLAY))
    column_tuples.append(('Total', LAST_YEAR_DISPLAY))
    column_tuples.append(('Total', 'Target'))
    column_tuples.append(('Total', CURRENT_DISPLAY))
    column_tuples.append(('Total', 'v/s Target %'))
    column_tuples.append(('Total', 'v/s Last Year %'))
    column_tuples.append(('Total', 'v/s Last Month %'))
    
    multi_columns = pd.MultiIndex.from_tuples(column_tuples, names=['Week', 'Period'])
    
    # Create the final dataframe
    final_df = pd.DataFrame(index=all_idgs, columns=multi_columns)
    
    # Fill the dataframe with data and calculate comparisons
    for idg in all_idgs:
        for week in weeks:
            week_label = get_week_label(week)
            
            # Fill period data
            for period in periods:
                if period in sheet_data and idg in sheet_data[period].index and week in sheet_data[period].columns:
                    final_df.loc[idg, (week_label, period)] = sheet_data[period].loc[idg, week]
                else:
                    final_df.loc[idg, (week_label, period)] = 0
            
            # Fill target data
            if not target_by_week.empty and week in target_by_week.index and idg in target_by_week.columns:
                final_df.loc[idg, (week_label, 'Target')] = target_by_week.loc[week, idg]
            else:
                final_df.loc[idg, (week_label, 'Target')] = 0
            
            # Calculate comparison percentages for each IDG and week
            current_val = final_df.loc[idg, (week_label, CURRENT_DISPLAY)]
            last_year_val = final_df.loc[idg, (week_label, LAST_YEAR_DISPLAY)]
            last_month_val = final_df.loc[idg, (week_label, LAST_MONTH_DISPLAY)]
            target_val = final_df.loc[idg, (week_label, 'Target')]
            
            # v/s Target %
            if target_val != 0:
                vs_target = (current_val / target_val * 100)  # Changed formula
                final_df.loc[idg, (week_label, 'v/s Target %')] = round(vs_target, 2)
            else:
                final_df.loc[idg, (week_label, 'v/s Target %')] = 0 if current_val == 0 else float('inf') # Or handle as per requirement for 0 target
            
            # v/s Last Year %
            if last_year_val != 0:
                vs_last_year = ((current_val - last_year_val) / last_year_val * 100)
                final_df.loc[idg, (week_label, 'v/s Last Year %')] = round(vs_last_year, 2)
            else:
                final_df.loc[idg, (week_label, 'v/s Last Year %')] = 0 if current_val == 0 else float('inf')
            
            # v/s Last Month %
            if last_month_val != 0:
                vs_last_month = ((current_val - last_month_val) / last_month_val * 100)
                final_df.loc[idg, (week_label, 'v/s Last Month %')] = round(vs_last_month, 2)
            else:
                final_df.loc[idg, (week_label, 'v/s Last Month %')] = 0 if current_val == 0 else float('inf')
    
    # Fill NaN values with 0
    final_df = final_df.fillna(0)
    
    # Calculate Total columns for each IDG
    for idg in all_idgs:
        # Calculate totals for each period across all weeks
        for period in periods:
            period_cols = [col for col in final_df.columns if col[1] == period and col[0] != 'Total']
            total_value = final_df.loc[idg, period_cols].sum()
            final_df.loc[idg, ('Total', period)] = total_value
        
        # Calculate total for target
        target_cols = [col for col in final_df.columns if col[1] == 'Target' and col[0] != 'Total']
        total_target = final_df.loc[idg, target_cols].sum()
        final_df.loc[idg, ('Total', 'Target')] = total_target
        
        # Calculate total comparison percentages
        total_current = final_df.loc[idg, ('Total', CURRENT_DISPLAY)]
        total_last_year = final_df.loc[idg, ('Total', LAST_YEAR_DISPLAY)]
        total_last_month = final_df.loc[idg, ('Total', LAST_MONTH_DISPLAY)]
        total_target = final_df.loc[idg, ('Total', 'Target')]
        
        # Total v/s Target %
        if total_target != 0:
            total_vs_target = (total_current / total_target * 100)  # Changed formula
            final_df.loc[idg, ('Total', 'v/s Target %')] = round(total_vs_target, 2)
        else:
            final_df.loc[idg, ('Total', 'v/s Target %')] = 0 if total_current == 0 else float('inf') # Or handle as per requirement for 0 target
        
        # Total v/s Last Year %
        if total_last_year != 0:
            total_vs_last_year = ((total_current - total_last_year) / total_last_year * 100)
            final_df.loc[idg, ('Total', 'v/s Last Year %')] = round(total_vs_last_year, 2)
        else:
            final_df.loc[idg, ('Total', 'v/s Last Year %')] = 0 if total_current == 0 else float('inf')
        
        # Total v/s Last Month %
        if total_last_month != 0:
            total_vs_last_month = ((total_current - total_last_month) / total_last_month * 100)
            final_df.loc[idg, ('Total', 'v/s Last Month %')] = round(total_vs_last_month, 2)
        else:
            final_df.loc[idg, ('Total', 'v/s Last Month %')] = 0 if total_current == 0 else float('inf')
    
    # Add Total row for all IDGs combined
    total_row_data = {}
    for col in final_df.columns:
        if 'v/s' in col[1]:  # For percentage columns, calculate weighted averages
            if col[1] in ['v/s Target %', 'v/s Last Year %', 'v/s Last Month %']:
                # Calculate overall percentage for the total row
                if col[0] == 'Total':  # Total column
                    total_current = final_df[('Total', CURRENT_DISPLAY)].sum()
                    total_base = 0
                    if col[1] == 'v/s Target %':
                        total_base = final_df[('Total', 'Target')].sum()
                    elif col[1] == 'v/s Last Year %':
                        total_base = final_df[('Total', LAST_YEAR_DISPLAY)].sum()
                    else:  # v/s Last Month %
                        total_base = final_df[('Total', LAST_MONTH_DISPLAY)].sum()
                    
                    if total_base != 0:
                        if col[1] == 'v/s Target %':
                            total_percentage = (total_current / total_base * 100) # Changed formula
                        else:
                            total_percentage = ((total_current - total_base) / total_base * 100)
                        total_row_data[col] = round(total_percentage, 2)
                    else:
                        total_row_data[col] = 0
                else:  # Week-wise percentage columns
                    week_label = col[0]
                    week_current = final_df[(week_label, CURRENT_DISPLAY)].sum()
                    week_base = 0
                    if col[1] == 'v/s Target %':
                        week_base = final_df[(week_label, 'Target')].sum()
                    elif col[1] == 'v/s Last Year %':
                        week_base = final_df[(week_label, LAST_YEAR_DISPLAY)].sum()
                    else:  # v/s Last Month %
                        week_base = final_df[(week_label, LAST_MONTH_DISPLAY)].sum()
                    
                    if week_base != 0:
                        if col[1] == 'v/s Target %':
                            week_percentage = (week_current / week_base * 100) # Changed formula
                        else:
                            week_percentage = ((week_current - week_base) / week_base * 100)
                        total_row_data[col] = round(week_percentage, 2)
                    else:
                        total_row_data[col] = 0
        else:
            # For amount columns, sum all IDGs
            total_row_data[col] = final_df[col].sum()
    
    # Create total row as a DataFrame and concatenate
    total_row_df = pd.DataFrame([total_row_data], index=['Total'])
    final_df = pd.concat([final_df, total_row_df])
    
    print(f"✅ {table_name} pivot table created successfully!")
    print(f"📊 Final shape: {final_df.shape}")
    
    return final_df, all_idgs, max_week

# Create three pivot tables: Overall, EA only, and Jumbo.ae only with Target Data
print("🔄 Creating Multiple Pivot Tables WITH TARGET DATA AND PROPER FILTERING...")
print("=" * 70)

# 1. Overall IDG Pivot Table (all data)
print("\n📊 1. OVERALL IDG ANALYSIS (All Types) - WITH TARGET DATA")
print("-" * 55)
global_idg_pivot, all_idgs_global, max_week_global = create_comprehensive_pivot_table(
    sheet_info, periods, type_filter=None, table_name="Overall IDG"
)

print(f"\n📊 Data Shape: {global_idg_pivot.shape}")
print(f"🏷️ IDG Categories: {len(all_idgs_global)} (+ Total row)")
print(f"📅 Periods: {periods} + Target + Comparisons")
print(f"📋 Weeks analyzed: {max_week_global} (Starting day: {start_day}) + Total column")

# Summary statistics by period for overall
print("\n📊 SUMMARY BY PERIOD (Overall):")
for period in periods:
    period_total = global_idg_pivot.loc['Total', ('Total', period)]
    print(f"  {period}: {period_total:,.2f}")
target_total = global_idg_pivot.loc['Total', ('Total', 'Target')]
print(f"  Target: {target_total:,.2f}")

print("\n📋 Overall IDG Pivot Table (with Target):")
display(global_idg_pivot.round(2))

# 2. EA Only Pivot Table
print("\n" + "=" * 70)
print("📊 2. EA ONLY ANALYSIS - WITH TARGET DATA")
print("-" * 45)
ea_idg_pivot, all_idgs_ea, max_week_ea = create_comprehensive_pivot_table(
    sheet_info, periods, type_filter="EA", table_name="EA IDG"
)

print(f"\n📊 Data Shape: {ea_idg_pivot.shape}")
print(f"🏷️ IDG Categories: {len(all_idgs_ea)} (+ Total row)")
print(f"📅 Periods: {periods} + Target + Comparisons")
print(f"📋 Weeks analyzed: {max_week_ea} (Starting day: {start_day}) + Total column")

# Summary statistics by period for EA
print("\n📊 SUMMARY BY PERIOD (EA Only):")
for period in periods:
    period_total = ea_idg_pivot.loc['Total', ('Total', period)]
    print(f"  {period}: {period_total:,.2f}")
target_total_ea = ea_idg_pivot.loc['Total', ('Total', 'Target')]
print(f"  Target: {target_total_ea:,.2f}")

print("\n📋 EA Only IDG Pivot Table (with Target):")
display(ea_idg_pivot.round(2))

# 3. Jumbo.ae Only Pivot Table
print("\n" + "=" * 70)
print("📊 3. JUMBO.AE ONLY ANALYSIS - WITH TARGET DATA")
print("-" * 50)
jumbo_idg_pivot, all_idgs_jumbo, max_week_jumbo = create_comprehensive_pivot_table(
    sheet_info, periods, type_filter="Jumbo.ae", table_name="Jumbo.ae IDG"
)

print(f"\n📊 Data Shape: {jumbo_idg_pivot.shape}")
print(f"🏷️ IDG Categories: {len(all_idgs_jumbo)} (+ Total row)")
print(f"📅 Periods: {periods} + Target + Comparisons")
print(f"📋 Weeks analyzed: {max_week_jumbo} (Starting day: {start_day}) + Total column")

# Summary statistics by period for Jumbo.ae
print("\n📊 SUMMARY BY PERIOD (Jumbo.ae Only):")
for period in periods:
    period_total = jumbo_idg_pivot.loc['Total', ('Total', period)]
    print(f"  {period}: {period_total:,.2f}")
target_total_jumbo = jumbo_idg_pivot.loc['Total', ('Total', 'Target')]
print(f"  Target: {target_total_jumbo:,.2f}")

print("\n📋 Jumbo.ae Only IDG Pivot Table (with Target):")
display(jumbo_idg_pivot.round(2))

print("\n" + "=" * 70)
print("✅ ALL PIVOT TABLES WITH TARGET DATA CREATED SUCCESSFULLY!")
print("📊 Summary:")
print(f"   • Overall Analysis: {len(all_idgs_global)} IDGs, {max_week_global} weeks")
print(f"   • EA Analysis: {len(all_idgs_ea)} IDGs, {max_week_ea} weeks")
print(f"   • Jumbo.ae Analysis: {len(all_idgs_jumbo)} IDGs, {max_week_jumbo} weeks")
print(f"   • Each table includes: {len(periods)} periods + Target + vs Target % + comparisons + totals")
print(f"   • Column order: Last Month, Last Year, Target, Current Month, vs Target %, vs Last Year %, vs Last Month %")
print("=" * 70)

In [ ]:
# IDG Analysis Report - Multiple Pivot Tables

# Additional pivot table views for all three analyses

# Weekly Analysis Views

# 1. Show first few weeks detailed view for Overall analysis
print("📊 OVERALL ANALYSIS - First 3 Weeks IDG Data")
print("=" * 50)
first_weeks_cols = []
for week in range(1, min(4, max_week_global + 1)):  # First 3 weeks or max available
    week_label = get_week_label(week)
    for period in periods:  # Use dynamic periods instead of hardcoded values
        first_weeks_cols.append((week_label, period))

if first_weeks_cols:  # Only if we have data
    first_weeks_data = global_idg_pivot[first_weeks_cols]
    display(first_weeks_data.round(2))
else:
    print("No week data available")

print("\n" + "=" * 50)

# 2. Weekly totals comparison for Overall analysis
print("📈 OVERALL ANALYSIS - Weekly Totals Comparison")
print("=" * 45)
weekly_totals = pd.DataFrame(index=range(1, max_week_global + 1), columns=periods)  # Use dynamic periods

for week in range(1, max_week_global + 1):
    week_label = get_week_label(week)
    for period in periods:  # Use dynamic periods
        week_period_cols = [(week_label, period)]
        weekly_total = global_idg_pivot[week_period_cols].sum().sum()
        weekly_totals.loc[week, period] = weekly_total

# Convert to numeric and add growth percentages
weekly_totals = weekly_totals.astype(float)
weekly_totals['Current_vs_LastYear_%'] = ((weekly_totals[CURRENT_DISPLAY] - weekly_totals[LAST_YEAR_DISPLAY]) / weekly_totals[LAST_YEAR_DISPLAY] * 100).round(2)
weekly_totals['Current_vs_LastMonth_%'] = ((weekly_totals[CURRENT_DISPLAY] - weekly_totals[LAST_MONTH_DISPLAY]) / weekly_totals[LAST_MONTH_DISPLAY] * 100).round(2)

# Add week labels for better readability
weekly_totals.index = [get_week_label(week) for week in range(1, max_week_global + 1)]

display(weekly_totals.round(2))

print("\n" + "=" * 50)

# 3. IDG totals by period (summary pivot) for Overall analysis
print("📋 OVERALL ANALYSIS - IDG Totals by Period (Week-based)")
print("=" * 55)
idg_period_summary = pd.DataFrame(index=all_idgs_global, columns=periods)  # Use dynamic periods

for idg in all_idgs_global:
    for period in periods:  # Use dynamic periods
        period_cols = [col for col in global_idg_pivot.columns if col[1] == period]
        idg_period_summary.loc[idg, period] = global_idg_pivot.loc[idg, period_cols].sum()

# Convert to numeric and add totals
idg_period_summary = idg_period_summary.astype(float)
idg_period_summary['Total'] = idg_period_summary.sum(axis=1)
idg_period_summary.loc['Total'] = idg_period_summary.sum()

display(idg_period_summary.round(2))

# 4. Quick comparison between the three analyses
print("\n" + "=" * 60)
print("📊 COMPARISON SUMMARY - All Three Analyses")
print("=" * 60)

# Create summary comparison
comparison_summary = pd.DataFrame({
    'Analysis_Type': ['Overall', 'EA Only', 'Jumbo.ae Only'],
    'IDG_Count': [len(all_idgs_global), len(all_idgs_ea), len(all_idgs_jumbo)],
    'Max_Weeks': [max_week_global, max_week_ea, max_week_jumbo]
})

# Add period totals for each analysis
for period in periods:
    comparison_summary[f'{period}_Total'] = [
        global_idg_pivot.loc['Total', ('Total', period)],
        ea_idg_pivot.loc['Total', ('Total', period)],
        jumbo_idg_pivot.loc['Total', ('Total', period)]
    ]

print("Summary of All Three Pivot Table Analyses:")
display(comparison_summary.round(2))

print("\n📈 Key Insights:")
print(f"• Overall analysis covers {len(all_idgs_global)} IDG categories across {max_week_global} weeks")
print(f"• EA analysis covers {len(all_idgs_ea)} IDG categories across {max_week_ea} weeks")
print(f"• Jumbo.ae analysis covers {len(all_idgs_jumbo)} IDG categories across {max_week_jumbo} weeks")
print(f"• Each analysis includes {len(periods)} periods: {', '.join(periods)}")
print("=" * 60)

In [ ]:
# Weekly Comparison Analysis - Focused View for All Three Analyses
print("📊 COMPREHENSIVE GROWTH COMPARISON ANALYSIS")
print("=" * 55)

def analyze_growth_patterns(pivot_table, all_idgs, max_week, analysis_name):
    """Analyze growth patterns for a given pivot table"""
    print(f"\n🔍 {analysis_name.upper()} ANALYSIS - Growth Patterns")
    print("-" * 50)
    
    # Create a focused view showing only comparison percentages
    comparison_cols = []
    for week in range(1, max_week + 1):
        week_label = get_week_label(week)
        comparison_cols.extend([
            (week_label, 'v/s Last Year %'),
            (week_label, 'v/s Last Month %')
        ])
    
    if comparison_cols:
        comparison_data = pivot_table[comparison_cols]
        print(f"\n📈 {analysis_name} - Growth Percentages by Week:")
        display(comparison_data.round(2))
        
        # Calculate average growth rates
        print(f"\n📊 {analysis_name} - Average Growth Rates Across All Weeks:")
        print("-" * 45)
        
        # Get all v/s Last Year columns
        last_year_cols = [col for col in comparison_data.columns if 'v/s Last Year %' in col[1]]
        last_month_cols = [col for col in comparison_data.columns if 'v/s Last Month %' in col[1]]
        
        for idg in all_idgs:
            avg_vs_last_year = comparison_data.loc[idg, last_year_cols].replace([float('inf'), -float('inf')], 0).mean()
            avg_vs_last_month = comparison_data.loc[idg, last_month_cols].replace([float('inf'), -float('inf')], 0).mean()
            print(f"{idg:15s} | Avg vs Last Year: {avg_vs_last_year:6.1f}% | Avg vs Last Month: {avg_vs_last_month:6.1f}%")
        
        # Weekly summary of overall performance
        print(f"\n📋 {analysis_name} - Weekly Performance Summary (All IDGs Combined):")
        print("-" * 55)
        
        weekly_performance = pd.DataFrame(index=range(1, max_week + 1), 
                                        columns=['vs_Last_Year_%', 'vs_Last_Month_%'])
        
        for week in range(1, max_week + 1):
            week_label = get_week_label(week)
            
            # Calculate total amounts for the week using dynamic period names
            current_total = pivot_table[(week_label, CURRENT_DISPLAY)].sum()
            last_year_total = pivot_table[(week_label, LAST_YEAR_DISPLAY)].sum()
            last_month_total = pivot_table[(week_label, LAST_MONTH_DISPLAY)].sum()
            
            # Calculate overall percentage changes
            if last_year_total != 0:
                vs_last_year = ((current_total - last_year_total) / last_year_total * 100)
                weekly_performance.loc[week, 'vs_Last_Year_%'] = round(vs_last_year, 2)
            else:
                weekly_performance.loc[week, 'vs_Last_Year_%'] = 0
                
            if last_month_total != 0:
                vs_last_month = ((current_total - last_month_total) / last_month_total * 100)
                weekly_performance.loc[week, 'vs_Last_Month_%'] = round(vs_last_month, 2)
            else:
                weekly_performance.loc[week, 'vs_Last_Month_%'] = 0
        
        # Add week labels for better readability
        weekly_performance.index = [get_week_label(week) for week in range(1, max_week + 1)]
        
        display(weekly_performance)
        
        return weekly_performance
    else:
        print(f"No comparison data available for {analysis_name}")
        return None

# Analyze all three pivot tables
overall_performance = analyze_growth_patterns(global_idg_pivot, all_idgs_global, max_week_global, "Overall")
ea_performance = analyze_growth_patterns(ea_idg_pivot, all_idgs_ea, max_week_ea, "EA Only")
jumbo_performance = analyze_growth_patterns(jumbo_idg_pivot, all_idgs_jumbo, max_week_jumbo, "Jumbo.ae Only")

# Cross-analysis comparison
print("\n" + "=" * 70)
print("🔄 CROSS-ANALYSIS PERFORMANCE COMPARISON")
print("=" * 70)

if overall_performance is not None and ea_performance is not None and jumbo_performance is not None:
    # Compare average performance across all analyses
    cross_comparison = pd.DataFrame({
        'Analysis': ['Overall', 'EA Only', 'Jumbo.ae Only'],
        'Avg_Growth_vs_Last_Year_%': [
            overall_performance['vs_Last_Year_%'].mean(),
            ea_performance['vs_Last_Year_%'].mean(),
            jumbo_performance['vs_Last_Year_%'].mean()
        ],
        'Avg_Growth_vs_Last_Month_%': [
            overall_performance['vs_Last_Month_%'].mean(),
            ea_performance['vs_Last_Month_%'].mean(),
            jumbo_performance['vs_Last_Month_%'].mean()
        ]
    })
    
    print("📊 Average Growth Rates Comparison:")
    display(cross_comparison.round(2))
    
    # Best and worst performing analysis types
    best_vs_year = cross_comparison.loc[cross_comparison['Avg_Growth_vs_Last_Year_%'].idxmax(), 'Analysis']
    best_vs_month = cross_comparison.loc[cross_comparison['Avg_Growth_vs_Last_Month_%'].idxmax(), 'Analysis']
    
    print(f"\n✅ Performance Highlights:")
    print(f"• Best performing vs Last Year: {best_vs_year}")
    print(f"• Best performing vs Last Month: {best_vs_month}")
else:
    print("Could not complete cross-analysis comparison")

print("\n" + "=" * 70)
print("✅ COMPREHENSIVE GROWTH ANALYSIS COMPLETE")
print("📊 All three pivot tables analyzed for growth patterns")
print("📈 Weekly performance trends calculated and compared")
print("=" * 70)

In [ ]:
# Export All IDG Pivot Tables to Excel - Combined in Single Sheet
print("📤 EXPORTING ALL THREE PIVOT TABLES TO SINGLE EXCEL SHEET...")
print("=" * 65)

# Create Excel writer object
output_file = 'IDG_Weekly_Analysis_Combined.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    
    # Create combined sheet with all three analyses
    print("📊 Creating Combined IDG Analysis Sheet...")
    
    # Create a workbook and worksheet - ensure we have a proper worksheet
    workbook = writer.book
    # Remove default sheet if it exists
    if workbook.worksheets:
        workbook.remove(workbook.active)
    
    # Create our main worksheet
    worksheet = workbook.create_sheet('Combined_IDG_Analysis', 0)
    
    # Define styles for different sections
    main_title_font = Font(bold=True, size=14, color='FFFFFF')
    main_title_fill = PatternFill(start_color='1F4E79', end_color='1F4E79', fill_type='solid')
    
    section_header_font = Font(bold=True, size=12, color='FFFFFF')
    section_header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
    
    column_header_font = Font(bold=True, size=10, color='FFFFFF')
    column_header_fill = PatternFill(start_color='4472C4', end_color='4472C4', fill_type='solid')
    
    idg_name_font = Font(bold=True, size=9)
    idg_name_fill = PatternFill(start_color='F2F2F2', end_color='F2F2F2', fill_type='solid')
    
    number_format = '#,##0.00'
    thin_border = Border(
        left=Side(style='thin'), right=Side(style='thin'),
        top=Side(style='thin'), bottom=Side(style='thin')
    )
    
    # MODIFIED Function to write a single pivot table to a dedicated worksheet
    def write_pivot_table_to_sheet(pivot_df, analysis_title, worksheet):
        # Section title for the sheet
        worksheet.cell(row=1, column=1, value=analysis_title)
        title_cell = worksheet.cell(row=1, column=1)
        title_cell.font = section_header_font # Use section_header_font for individual sheet titles
        title_cell.fill = section_header_fill
        title_cell.alignment = Alignment(horizontal='center', vertical='center')
        
        # Merge cells for the title across all columns of the pivot table
        num_cols = len(pivot_df.columns)
        worksheet.merge_cells(start_row=1, start_column=1, 
                            end_row=1, end_column=num_cols + 1) # +1 for the IDG column
        
        # Apply border to title
        for col_idx_merge in range(1, num_cols + 2):
            worksheet.cell(row=1, column=col_idx_merge).border = thin_border
        
        current_row = 2 # Start headers from row 2
        
        # Write column headers (multi-level)
        # First level headers (Week labels)
        col_idx = 2  # Start from column 2 (column 1 is for IDG names)
        for week_label in pivot_df.columns.get_level_values(0).unique():
            week_cols = [col for col in pivot_df.columns if col[0] == week_label]
            num_sub_cols = len(week_cols)
            
            worksheet.cell(row=current_row, column=col_idx, value=week_label)
            week_cell = worksheet.cell(row=current_row, column=col_idx)
            week_cell.font = column_header_font
            week_cell.fill = column_header_fill
            week_cell.alignment = Alignment(horizontal='center', vertical='center')
            week_cell.border = thin_border
            
            if num_sub_cols > 1:
                worksheet.merge_cells(start_row=current_row, start_column=col_idx,
                                    end_row=current_row, end_column=col_idx + num_sub_cols - 1)
                for i in range(num_sub_cols):
                    cell = worksheet.cell(row=current_row, column=col_idx + i)
                    cell.font = column_header_font
                    cell.fill = column_header_fill
                    cell.border = thin_border
            col_idx += num_sub_cols
        
        current_row += 1
        
        # Second level headers (Period names)
        worksheet.cell(row=current_row, column=1, value="IDG")
        idg_header_cell = worksheet.cell(row=current_row, column=1)
        idg_header_cell.font = column_header_font
        idg_header_cell.fill = column_header_fill
        idg_header_cell.alignment = Alignment(horizontal='center', vertical='center')
        idg_header_cell.border = thin_border
        
        col_idx = 2
        for col_multi_idx in pivot_df.columns:
            worksheet.cell(row=current_row, column=col_idx, value=col_multi_idx[1])
            sub_header_cell = worksheet.cell(row=current_row, column=col_idx)
            sub_header_cell.font = column_header_font
            sub_header_cell.fill = column_header_fill
            sub_header_cell.alignment = Alignment(horizontal='center', vertical='center')
            sub_header_cell.border = thin_border
            col_idx += 1
        
        current_row += 1
        
        # Write data rows
        for idg_name in pivot_df.index:
            worksheet.cell(row=current_row, column=1, value=idg_name)
            name_cell = worksheet.cell(row=current_row, column=1)
            if idg_name == 'Total':
                name_cell.font = Font(bold=True, size=10)
                name_cell.fill = PatternFill(start_color='FFE699', end_color='FFE699', fill_type='solid')
            else:
                name_cell.font = idg_name_font
                name_cell.fill = idg_name_fill
            name_cell.alignment = Alignment(horizontal='left', vertical='center')
            name_cell.border = thin_border
            
            col_idx = 2
            for col_multi_idx in pivot_df.columns:
                value = pivot_df.loc[idg_name, col_multi_idx]
                worksheet.cell(row=current_row, column=col_idx, value=value)
                data_cell = worksheet.cell(row=current_row, column=col_idx)
                
                if idg_name == 'Total':
                    data_cell.font = Font(bold=True, size=9)
                    data_cell.fill = PatternFill(start_color='FFE699', end_color='FFE699', fill_type='solid')
                else:
                    data_cell.font = Font(size=9)
                
                if 'v/s' in col_multi_idx[1] and '%' in col_multi_idx[1]:
                    data_cell.number_format = '0.00"%"'
                    if isinstance(value, (int, float)) and value != float('inf') and value != float('-inf') and value != 0 : # Check for valid numeric value
                        if col_multi_idx[1] == 'v/s Target %':
                            if value >= 100:
                                data_cell.font = Font(color='008000', size=9, bold=idg_name=='Total')
                            else:
                                data_cell.font = Font(color='FF0000', size=9, bold=idg_name=='Total')
                        else:
                            if value > 0:
                                data_cell.font = Font(color='008000', size=9, bold=idg_name=='Total')
                            elif value < 0:
                                data_cell.font = Font(color='FF0000', size=9, bold=idg_name=='Total')
                else:
                    data_cell.number_format = number_format
                
                data_cell.alignment = Alignment(horizontal='right', vertical='center')
                data_cell.border = thin_border
                col_idx += 1
            current_row += 1
            
        # Auto-adjust column widths for this worksheet
        for col_letter_idx in range(1, worksheet.max_column + 1):
            max_length = 0
            column_letter = get_column_letter(col_letter_idx)
            for row_idx in range(1, worksheet.max_row + 1):
                try:
                    cell_value = str(worksheet.cell(row=row_idx, column=col_letter_idx).value)
                    if len(cell_value) > max_length:
                        max_length = len(cell_value)
                except:
                    pass
            adjusted_width = min(max_length + 2, 30) # Max width of 30, increased padding
            worksheet.column_dimensions[column_letter].width = adjusted_width

        # Freeze panes for this worksheet (IDG name and headers)
        # Freeze row 3 (headers) and column 1 (IDG names)
        worksheet.freeze_panes = 'B1' # Top 3 rows and 1st column frozen

    # Main Excel writing block
    # Remove default sheet if it exists
    if workbook.worksheets:
        workbook.remove(workbook.active)
    
    # Data for sheets
    analyses_to_export = [
        {"name": "Overall_IDG_Analysis", "title": f"OVERALL IDG ANALYSIS ({max_invoice_day} days)", "data": global_idg_pivot},
        {"name": "EA_IDG_Analysis", "title": f"EA ONLY IDG ANALYSIS ({max_invoice_day} days)", "data": ea_idg_pivot},
        {"name": "JumboAE_IDG_Analysis", "title": f"JUMBO.AE ONLY IDG ANALYSIS ({max_invoice_day} days)", "data": jumbo_idg_pivot},
    ]

    sheet_idx_counter = 0
    for analysis in analyses_to_export:
        print(f"  📊 Writing {analysis['title']} to sheet: {analysis['name']}...")
        ws = workbook.create_sheet(analysis['name'], sheet_idx_counter)
        write_pivot_table_to_sheet(analysis['data'].round(2), analysis['title'], ws)
        sheet_idx_counter += 1
    
    # Create additional supporting sheets (Summary Dashboard, Weekly Totals)
    # These will be created after the individual analysis sheets.
    
    # 2. Summary Dashboard Sheet
    print("📊 Creating Summary Dashboard...")
    summary_data = {
        'Analysis_Type': ['Overall', 'EA Only', 'Jumbo.ae Only'],
        'IDG_Categories': [len(all_idgs_global), len(all_idgs_ea), len(all_idgs_jumbo)],
        'Weeks_Analyzed': [max_week_global, max_week_ea, max_week_jumbo]
    }
    
    # Add period totals for each analysis
    for period in periods:
        summary_data[f'{period}_Total'] = [
            global_idg_pivot.loc['Total', ('Total', period)],
            ea_idg_pivot.loc['Total', ('Total', period)],
            jumbo_idg_pivot.loc['Total', ('Total', period)]
        ]
    
    # Add growth percentages
    summary_data[f'Growth_vs_{LAST_YEAR_DISPLAY}_%'] = [
        global_idg_pivot.loc['Total', ('Total', 'v/s Last Year %')],
        ea_idg_pivot.loc['Total', ('Total', 'v/s Last Year %')],
        jumbo_idg_pivot.loc['Total', ('Total', 'v/s Last Year %')]
    ]
    
    summary_data[f'Growth_vs_{LAST_MONTH_DISPLAY}_%'] = [
        global_idg_pivot.loc['Total', ('Total', 'v/s Last Month %')],
        ea_idg_pivot.loc['Total', ('Total', 'v/s Last Month %')],
        jumbo_idg_pivot.loc['Total', ('Total', 'v/s Last Month %')]
    ]
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.round(2).to_excel(writer, sheet_name='Summary_Dashboard', index=False)
    
    # 3. Weekly Totals Summary (Overall)
    print("📊 Creating Weekly Totals Summary...")
    weekly_totals = pd.DataFrame(index=range(1, max_week_global + 1), columns=periods)
    
    for week in range(1, max_week_global + 1):
        week_label = get_week_label(week)
        for period in periods:
            week_period_cols = [(week_label, period)]
            weekly_total = global_idg_pivot[week_period_cols].sum().sum()
            weekly_totals.loc[week, period] = weekly_total
    
    # Add growth percentages using dynamic period names
    weekly_totals = weekly_totals.astype(float)
    weekly_totals['Growth_vs_LastYear_%'] = ((weekly_totals[CURRENT_DISPLAY] - weekly_totals[LAST_YEAR_DISPLAY]) / weekly_totals[LAST_YEAR_DISPLAY] * 100).round(2)
    weekly_totals['Growth_vs_LastMonth_%'] = ((weekly_totals[CURRENT_DISPLAY] - weekly_totals[LAST_MONTH_DISPLAY]) / weekly_totals[LAST_MONTH_DISPLAY] * 100).round(2)
    
    # Add week labels
    weekly_totals.index = [get_week_label(week) for week in range(1, max_week_global + 1)]
    
    # Export weekly totals
    weekly_totals.round(2).to_excel(writer, sheet_name='Weekly_Totals', index=True)

print(f"\n✅ EXCEL EXPORT COMPLETED SUCCESSFULLY!")
print(f"📎 File: {output_file}")
print("\n📊 SHEETS CREATED:")
print("=" * 50)
print("📄 1. Overall_IDG_Analysis - Overall IDG Analysis table")
print("📄 2. EA_IDG_Analysis - EA Only IDG Analysis table")
print("📄 3. JumboAE_IDG_Analysis - Jumbo.ae Only IDG Analysis table")
print(f"📄 4. Summary_Dashboard - Overview of all analyses") # Adjusted numbering
print(f"📄 5. Weekly_Totals - Weekly summary with growth")   # Adjusted numbering

print(f"\n🎆 FEATURES APPLIED TO EACH ANALYSIS SHEET:")
print(f"• Dedicated sheet for each analysis: Overall, EA, Jumbo.ae")
print(f"• Professional multi-level header formatting")
print(f"• Color-coded percentages (Green: positive, Red: negative)")
print(f"• Highlighted Total columns and rows")
print(f"• Auto-adjusted column widths for readability")
print(f"• Frozen panes for easy navigation")
print(f"• Consistent number formatting across all sheets")
print(f"• Comprehensive analysis covering all three data types")

print(f"\n📊 ANALYSIS COVERAGE:")
print(f"• Overall: {len(all_idgs_global)} IDGs across {max_week_global} weeks")
print(f"• EA Only: {len(all_idgs_ea)} IDGs across {max_week_ea} weeks")
print(f"• Jumbo.ae: {len(all_idgs_jumbo)} IDGs across {max_week_jumbo} weeks")
print(f"• Periods: {', '.join(periods)}")
print(f"• Week calculation based on: {start_day} as first day")

print(f"\n💾 File saved as: {output_file}")
print("=" * 60)

In [ ]:
# Dynamic constants for sessions
LAST_MONTH_SESSION_PATH = sessions_info[0][0]
LAST_MONTH_SESSION_SHEET = sessions_info[0][1]
LAST_MONTH_SESSION_DISPLAY = sessions_info[0][2]

LAST_YEAR_SESSION_PATH = sessions_info[1][0]
LAST_YEAR_SESSION_SHEET = sessions_info[1][1]
LAST_YEAR_SESSION_DISPLAY = sessions_info[1][2]

CURRENT_SESSION_PATH = sessions_info[2][0]
CURRENT_SESSION_SHEET = sessions_info[2][1]
CURRENT_SESSION_DISPLAY = sessions_info[2][2]

# Metrics to include in the pivot table
METRICS_TO_AGGREGATE = ['Sessions', 'Purchases', 'Purchase revenue']

def create_master_sessions_pivot(sessions_info, cg_filter=None):
    """
    Create master sessions pivot table with optional CG column filtering
    
    Parameters:
    sessions_info: List of tuples containing (path, sheet, display_name)
    cg_filter: String indicating filter type:
               - None: No filter (default)
               - "EA_only": Only include "EA" or "Endless Aisle" 
               - "non_EA": Exclude "EA" or "Endless Aisle"
    
    Returns:
    pd.DataFrame: Master pivot table with calculated metrics
    """
    
    print("📊 SESSIONS DATA ANALYSIS - WEEKLY MASTER PIVOT")
    if cg_filter == "EA_only":
        print("🔍 Filter: EA/Endless Aisle ONLY")
    elif cg_filter == "non_EA":
        print("🔍 Filter: NON-EA/Endless Aisle")
    else:
        print("🔍 Filter: NO FILTER")
    print("=" * 60)

    processed_sessions_data = {}
    all_channels = set()
    max_week_overall = 0

    # Read and process sessions data from all three periods
    for i, (path, sheet, display_name) in enumerate(sessions_info):
        print(f"\n📄 Processing {display_name} Sessions Data:")
        print("-" * 40)

        try:
            # Read the sessions data
            sessions_df = pd.read_excel(path, sheet_name=sheet)
            print(f"  Raw Shape: {sessions_df.shape}")
            print(f"  Raw Columns: {list(sessions_df.columns)}")
            
            # Filter out "Gift Card" from Category column if it exists
            if 'Category' in sessions_df.columns:
                sessions_df = sessions_df[sessions_df['Category'] != 'Gift Card ']
                print(f"  Shape after filtering 'Gift Card': {sessions_df.shape}")
            
            # Apply CG filter if specified
            if cg_filter and 'CG' in sessions_df.columns:
                if cg_filter == "EA_only":
                    sessions_df = sessions_df[sessions_df['CG'].isin(['EA', 'Endless Aisle'])]
                    print(f"  Shape after EA filter: {sessions_df.shape}")
                elif cg_filter == "non_EA":
                    sessions_df = sessions_df[~sessions_df['CG'].isin(['EA', 'Endless Aisle'])]
                    print(f"  Shape after non-EA filter: {sessions_df.shape}")
            elif cg_filter and 'CG' not in sessions_df.columns:
                print(f"  Warning: CG column not found, filter '{cg_filter}' cannot be applied")
            
            # Convert the string to datetime format first
            sessions_df['Date'] = pd.to_datetime(sessions_df['Date'], format='%Y%m%d', errors='coerce')

            # Then extract the day
            sessions_df['Day'] = sessions_df['Date'].dt.day
            


            sessions_df = sessions_df.dropna(subset=['Day']) 
            if sessions_df.empty:
                print(f"  No valid 'Day' data after conversion for {display_name}.")
                processed_sessions_data[display_name] = pd.DataFrame()
                continue

            sessions_df['Day'] = sessions_df['Day'].astype(int)

            original_len = len(sessions_df)
            sessions_df = sessions_df[sessions_df['Day'] <= max_invoice_day]
            print(f"📉 {display_name}: Filtered {original_len - len(sessions_df)} rows with Day > {max_invoice_day}")

            # Add WeekNumber column (ensure get_week_number and first_day_position are defined in a previous cell)
            sessions_df['WeekNumber'] = sessions_df['Day'].apply(
                lambda day: get_week_number(day, first_day_position) 
            )
            
            # Check which of the desired metrics are available in the current DataFrame
            available_metrics = [m for m in METRICS_TO_AGGREGATE if m in sessions_df.columns]
            if not available_metrics:
                print(f"  No metrics ({', '.join(METRICS_TO_AGGREGATE)}) found in {display_name}. Skipping.")
                processed_sessions_data[display_name] = pd.DataFrame()
                continue
            
            print(f"  Available metrics for {display_name}: {available_metrics}")

            # Select relevant columns for aggregation
            cols_for_aggregation = ['Channel', 'WeekNumber'] + available_metrics
            temp_df = sessions_df[cols_for_aggregation].copy()

            # Group by Channel and WeekNumber, and sum the available metrics
            aggregated_data_for_period = temp_df.groupby(['Channel', 'WeekNumber'])[available_metrics].sum().fillna(0)
            
            processed_sessions_data[display_name] = aggregated_data_for_period
            if not aggregated_data_for_period.empty:
                all_channels.update(aggregated_data_for_period.index.get_level_values('Channel').unique())
                # Ensure WeekNumber exists in index before calling max()
                if 'WeekNumber' in aggregated_data_for_period.index.names:
                    current_max_week_in_period = aggregated_data_for_period.index.get_level_values('WeekNumber').max()
                    if pd.notna(current_max_week_in_period) and current_max_week_in_period > max_week_overall:
                        max_week_overall = int(current_max_week_in_period)
                else: # Handle case where WeekNumber might not be in index (e.g. if groupby results in empty df for some reason)
                     current_max_week_in_period = 0
            else:
                current_max_week_in_period = 0
            
            print(f"  Processed {display_name} successfully. Max week: {current_max_week_in_period if current_max_week_in_period > 0 else 'N/A'}")

        except Exception as e:
            print(f"❌ Error processing {display_name}: {e}")
            import traceback
            traceback.print_exc()
            processed_sessions_data[display_name] = pd.DataFrame()

    print("\n" + "=" * 60)
    print("📊 CREATING FINAL COMPREHENSIVE SESSIONS MASTER PIVOT TABLE")

    if not processed_sessions_data or not all_channels:
        print("⚠️ No session data successfully processed or no channels found. Cannot create master pivot table.")
        return pd.DataFrame()
    
    sorted_channels = sorted(list(all_channels))
    session_periods_display_names = [LAST_MONTH_SESSION_DISPLAY, LAST_YEAR_SESSION_DISPLAY, CURRENT_SESSION_DISPLAY]
    
    column_tuples = []
    if max_week_overall > 0:
        weeks_for_pivot = list(range(1, int(max_week_overall) + 1))
        for week_num in weeks_for_pivot:
            week_label = get_week_label(week_num) # Ensure get_week_label is available
            for metric_name in METRICS_TO_AGGREGATE: # Use the full list of desired metrics
                for period_display_name in session_periods_display_names:
                    column_tuples.append((week_label, metric_name, period_display_name))
    
        if not column_tuples:
             print("⚠️ No weeks or metrics to create columns for. Pivot table will be empty or incomplete.")
             master_sessions_pivot_df = pd.DataFrame(index=sorted_channels)
        else:
            multi_columns = pd.MultiIndex.from_tuples(column_tuples, names=['Week', 'Metric', 'Period'])
            master_sessions_pivot_df = pd.DataFrame(index=sorted_channels, columns=multi_columns)
            master_sessions_pivot_df = master_sessions_pivot_df.fillna(0) # Initialize with 0

            for channel_val in sorted_channels:
                for week_num in weeks_for_pivot:
                    week_label = get_week_label(week_num)
                    for period_display_name in session_periods_display_names:
                        if period_display_name in processed_sessions_data:
                            period_aggregated_data = processed_sessions_data[period_display_name]
                            if not period_aggregated_data.empty and (channel_val, week_num) in period_aggregated_data.index:
                                data_series_for_channel_week = period_aggregated_data.loc[(channel_val, week_num)]
                                for metric_name in METRICS_TO_AGGREGATE:
                                    if metric_name in data_series_for_channel_week.index: # Check if metric was available for this period
                                        value = data_series_for_channel_week[metric_name]
                                        master_sessions_pivot_df.loc[channel_val, (week_label, metric_name, period_display_name)] = value
    else:
        print("⚠️ No weeks found in session data across all periods. Cannot create master pivot table.")
        return pd.DataFrame()

    # Calculate CVR and AOV if the DataFrame is not empty and has the required structure
    if not master_sessions_pivot_df.empty and isinstance(master_sessions_pivot_df.columns, pd.MultiIndex) and master_sessions_pivot_df.columns.nlevels == 3:

        # Collect unique (week_label, period_name) combinations that have base metrics
        processed_combinations = set()
        for col_tuple in master_sessions_pivot_df.columns:
            week_label, metric, period_name = col_tuple
            if metric in METRICS_TO_AGGREGATE:
                processed_combinations.add((week_label, period_name))

        for week_label, period_name in processed_combinations:
            sessions_col_tuple = (week_label, 'Sessions', period_name)
            purchases_col_tuple = (week_label, 'Purchases', period_name)
            revenue_col_tuple = (week_label, 'Purchase revenue', period_name)

            # Check if all base metric columns exist for this combination
            if not (sessions_col_tuple in master_sessions_pivot_df.columns and \
                    purchases_col_tuple in master_sessions_pivot_df.columns and \
                    revenue_col_tuple in master_sessions_pivot_df.columns):
                continue

            sessions_s = master_sessions_pivot_df[sessions_col_tuple]
            purchases_s = master_sessions_pivot_df[purchases_col_tuple]
            purchase_revenue_s = master_sessions_pivot_df[revenue_col_tuple]

            # Calculate CVR (Purchases / Sessions)
            cvr = purchases_s / sessions_s
            master_sessions_pivot_df[(week_label, 'CVR', period_name)] = cvr.fillna(0).replace([float('inf'), -float('inf')], 0)

            # Calculate AOV (Purchase Revenue / Purchases)
            aov = purchase_revenue_s / purchases_s
            master_sessions_pivot_df[(week_label, 'AOV', period_name)] = aov.fillna(0).replace([float('inf'), -float('inf')], 0)

        # Re-sort columns to ensure CVR and AOV are placed correctly and maintain overall structure
        all_metrics_ordered = METRICS_TO_AGGREGATE + ['CVR', 'AOV']
        
        # Get unique week labels and sort them
        unique_week_labels_sorted = sorted(list(master_sessions_pivot_df.columns.get_level_values('Week').unique()))

        # Get ordered periods
        period_order_from_sessions_info = [s_info[2] for s_info in sessions_info] 
        actual_periods_in_df = list(master_sessions_pivot_df.columns.get_level_values('Period').unique())
        ordered_periods = [p for p in period_order_from_sessions_info if p in actual_periods_in_df]
        for p_df in actual_periods_in_df:
            if p_df not in ordered_periods:
                ordered_periods.append(p_df)

        new_column_tuples = []
        for week_l in unique_week_labels_sorted:
            for metric_n in all_metrics_ordered:
                for period_dn in ordered_periods: 
                    # Check if this week/metric/period combination had any original data or newly added CVR/AOV
                    if (week_l, metric_n, period_dn) in master_sessions_pivot_df.columns:
                        new_column_tuples.append((week_l, metric_n, period_dn))
        
        if new_column_tuples:
            master_sessions_pivot_df = master_sessions_pivot_df.reindex(columns=pd.MultiIndex.from_tuples(new_column_tuples))
    
    # Add Grand Total row
    if not master_sessions_pivot_df.empty:
        print("📊 Adding Grand Total row...")
        
        # Create grand total row
        grand_total_row = pd.DataFrame(index=['Grand Total'], columns=master_sessions_pivot_df.columns)
        
        # Calculate totals for each column
        for col in master_sessions_pivot_df.columns:
            week_label, metric, period_name = col
            
            if metric in ['Sessions', 'Purchases', 'Purchase revenue']:
                # Sum these metrics
                grand_total_row.loc['Grand Total', col] = master_sessions_pivot_df[col].sum()
            elif metric == 'CVR':
                # Calculate overall CVR = Total Purchases / Total Sessions
                total_sessions_col = (week_label, 'Sessions', period_name)
                total_purchases_col = (week_label, 'Purchases', period_name)
                
                if total_sessions_col in master_sessions_pivot_df.columns and total_purchases_col in master_sessions_pivot_df.columns:
                    total_sessions = master_sessions_pivot_df[total_sessions_col].sum()
                    total_purchases = master_sessions_pivot_df[total_purchases_col].sum()
                    
                    if total_sessions > 0:
                        grand_total_row.loc['Grand Total', col] = total_purchases / total_sessions
                    else:
                        grand_total_row.loc['Grand Total', col] = 0
                else:
                    grand_total_row.loc['Grand Total', col] = 0
            elif metric == 'AOV':
                # Calculate overall AOV = Total Revenue / Total Purchases
                total_revenue_col = (week_label, 'Purchase revenue', period_name)
                total_purchases_col = (week_label, 'Purchases', period_name)
                
                if total_revenue_col in master_sessions_pivot_df.columns and total_purchases_col in master_sessions_pivot_df.columns:
                    total_revenue = master_sessions_pivot_df[total_revenue_col].sum()
                    total_purchases = master_sessions_pivot_df[total_purchases_col].sum()
                    
                    if total_purchases > 0:
                        grand_total_row.loc['Grand Total', col] = total_revenue / total_purchases
                    else:
                        grand_total_row.loc['Grand Total', col] = 0
                else:
                    grand_total_row.loc['Grand Total', col] = 0
        
        # Append grand total row to the main dataframe
        master_sessions_pivot_df = pd.concat([master_sessions_pivot_df, grand_total_row.fillna(0)])
        print("✅ Grand Total row added successfully")
    
    return master_sessions_pivot_df.fillna(0)

# Generate all three pivot tables
print("🚀 GENERATING ALL THREE PIVOT TABLES")
print("=" * 80)

# 1. No filter pivot
print("\n1️⃣ CREATING NO FILTER PIVOT TABLE")
master_sessions_pivot_no_filter = create_master_sessions_pivot(sessions_info, cg_filter=None)

# 2. EA only pivot
print("\n2️⃣ CREATING EA ONLY PIVOT TABLE")
master_sessions_pivot_ea_only = create_master_sessions_pivot(sessions_info, cg_filter="EA_only")

# 3. Non-EA pivot
print("\n3️⃣ CREATING NON-EA PIVOT TABLE")
master_sessions_pivot_non_ea = create_master_sessions_pivot(sessions_info, cg_filter="non_EA")

# Display all three pivot tables
print("\n" + "=" * 80)
print("📊 DISPLAYING ALL PIVOT TABLES")
print("=" * 80)

print("\n🔍 NO FILTER PIVOT TABLE:")
print("-" * 40)
display(master_sessions_pivot_no_filter)

print("\n🔍 EA ONLY PIVOT TABLE:")
print("-" * 40)
display(master_sessions_pivot_ea_only)

print("\n🔍 NON-EA PIVOT TABLE:")
print("-" * 40)
display(master_sessions_pivot_non_ea)

In [ ]:
# Export sessions pivot tables to Excel with professional formatting
print("📤 EXPORTING BEAUTIFULLY FORMATTED SESSIONS PIVOT TABLES TO EXCEL...")
print("=" * 80)

output_file = 'IDG_Weekly_Analysis_Combined.xlsx'

try:
    # Load existing workbook
    from openpyxl import load_workbook
    from openpyxl.styles import Font, PatternFill, Border, Side, Alignment, numbers
    from openpyxl.utils import get_column_letter
    wb = load_workbook(output_file)
    
    # Define styles for formatting consistency with previous tables
    section_header_font = Font(bold=True, size=14, color='FFFFFF')
    section_header_fill = PatternFill(start_color='1F4E79', end_color='1F4E79', fill_type='solid')
    
    column_header_font = Font(bold=True, size=10, color='FFFFFF')
    column_header_fill = PatternFill(start_color='4472C4', end_color='4472C4', fill_type='solid')
    
    subheader_font = Font(bold=True, size=10, color='FFFFFF')
    subheader_fill = PatternFill(start_color='5B9BD5', end_color='5B9BD5', fill_type='solid')
    
    row_header_font = Font(bold=True, size=9)
    row_header_fill = PatternFill(start_color='F2F2F2', end_color='F2F2F2', fill_type='solid')
    
    total_row_font = Font(bold=True, size=10)
    total_row_fill = PatternFill(start_color='FFE699', end_color='FFE699', fill_type='solid')
    
    thin_border = Border(
        left=Side(style='thin'), right=Side(style='thin'),
        top=Side(style='thin'), bottom=Side(style='thin')
    )
    
    def write_sessions_pivot_to_sheet(pivot_df, sheet_name):
        print(f"Writing beautifully formatted sessions data to sheet: {sheet_name}")
        
        if sheet_name in wb.sheetnames:
            # Get the existing sheet
            ws = wb[sheet_name]
            
            # Find the first empty row (assuming data starts from row 1)
            last_row = ws.max_row
            start_row = last_row + 3  # Leave two blank rows for spacing
            
            # Determine column spans
            total_columns = len(pivot_df.columns) + 1  # +1 for the row headers column
            
            # Write section header with professional formatting
            ws.cell(row=start_row, column=1, value="SESSIONS ANALYSIS")
            header_cell = ws.cell(row=start_row, column=1)
            header_cell.font = section_header_font
            header_cell.fill = section_header_fill
            header_cell.alignment = Alignment(horizontal='center', vertical='center')
            
            # Merge cells for the header
            ws.merge_cells(start_row=start_row, start_column=1, end_row=start_row, end_column=total_columns)
            
            # Apply borders to all merged header cells
            for col_idx in range(1, total_columns + 1):
                cell = ws.cell(row=start_row, column=col_idx)
                cell.border = thin_border
            
            # Track current row after header
            current_row = start_row + 1
            
            # Create true multi-level headers (3 levels: Week, Metric, Period)
            
            # First, organize the columns into a hierarchy: Week -> Metric -> Period
            header_hierarchy = {}
            for col in pivot_df.columns:
                week, metric, period = col
                if week not in header_hierarchy:
                    header_hierarchy[week] = {}
                if metric not in header_hierarchy[week]:
                    header_hierarchy[week][metric] = []
                header_hierarchy[week][metric].append(period)
            
            # Row 1: Week headers (top level)
            col_idx = 2  # Start from column 2 (column 1 is for row labels)
            week_start_columns = {}  # To track where each week starts
            
            for week in header_hierarchy:
                week_start_columns[week] = col_idx
                
                # Calculate total columns for this week
                week_total_cols = 0
                for metric in header_hierarchy[week]:
                    week_total_cols += len(header_hierarchy[week][metric])
                
                # Write week header and merge cells
                ws.cell(row=current_row, column=col_idx, value=week)
                week_cell = ws.cell(row=current_row, column=col_idx)
                week_cell.font = column_header_font
                week_cell.fill = column_header_fill
                week_cell.alignment = Alignment(horizontal='center', vertical='center')
                week_cell.border = thin_border
                
                # Merge cells if needed
                if week_total_cols > 1:
                    ws.merge_cells(start_row=current_row, start_column=col_idx, 
                                  end_row=current_row, end_column=col_idx + week_total_cols - 1)
                    
                    # Apply styles to all merged cells
                    for i in range(week_total_cols):
                        merged_cell = ws.cell(row=current_row, column=col_idx + i)
                        merged_cell.border = thin_border
                        merged_cell.font = column_header_font
                        merged_cell.fill = column_header_fill
                
                # Move to next position
                col_idx += week_total_cols
            
            # Add "Channel" header for first column
            ws.cell(row=current_row, column=1, value="Channel")
            channel_header = ws.cell(row=current_row, column=1)
            channel_header.font = column_header_font
            channel_header.fill = column_header_fill
            channel_header.alignment = Alignment(horizontal='center', vertical='center')
            channel_header.border = thin_border
            ws.merge_cells(start_row=current_row, start_column=1, 
                          end_row=current_row + 2, end_column=1)  # Merge across all 3 header rows
            
            # Row 2: Metric headers (middle level)
            current_row += 1
            
            for week in header_hierarchy:
                metric_start_col = week_start_columns[week]
                
                for metric in header_hierarchy[week]:
                    # Calculate how many columns this metric spans
                    metric_periods = header_hierarchy[week][metric]
                    metric_cols = len(metric_periods)
                    
                    # Write metric header
                    ws.cell(row=current_row, column=metric_start_col, value=metric)
                    metric_cell = ws.cell(row=current_row, column=metric_start_col)
                    metric_cell.font = column_header_font
                    metric_cell.fill = subheader_fill  # Slightly different shade than the week
                    metric_cell.alignment = Alignment(horizontal='center', vertical='center')
                    metric_cell.border = thin_border
                    
                    # Merge cells if needed
                    if metric_cols > 1:
                        ws.merge_cells(start_row=current_row, start_column=metric_start_col, 
                                      end_row=current_row, end_column=metric_start_col + metric_cols - 1)
                        
                        # Apply styles to all merged cells
                        for i in range(metric_cols):
                            merged_cell = ws.cell(row=current_row, column=metric_start_col + i)
                            merged_cell.border = thin_border
                            merged_cell.font = column_header_font
                            merged_cell.fill = subheader_fill
                    
                    # Move to next position
                    metric_start_col += metric_cols
            
            # Row 3: Period headers (bottom level)
            current_row += 1
            col_idx = 2  # Reset column index
            
            for col in pivot_df.columns:
                period = col[2]  # Third level
                
                # Write period header
                ws.cell(row=current_row, column=col_idx, value=period)
                period_cell = ws.cell(row=current_row, column=col_idx)
                period_cell.font = subheader_font
                period_cell.fill = PatternFill(start_color='B4C6E7', end_color='B4C6E7', fill_type='solid')  # Lighter blue
                period_cell.alignment = Alignment(horizontal='center', vertical='center')
                period_cell.border = thin_border
                
                col_idx += 1
            
            # Move to next row for data
            current_row += 1
            
            # Write data rows
            for idx in pivot_df.index:
                # Write row header (Channel name)
                ws.cell(row=current_row, column=1, value=str(idx))
                row_header_cell = ws.cell(row=current_row, column=1)
                
                # Special formatting for Grand Total
                if idx == "Grand Total":
                    row_header_cell.font = total_row_font
                    row_header_cell.fill = total_row_fill
                else:
                    row_header_cell.font = row_header_font
                    row_header_cell.fill = row_header_fill
                    
                row_header_cell.alignment = Alignment(horizontal='left', vertical='center')
                row_header_cell.border = thin_border
                
                # Write data values
                col_idx = 2
                for col in pivot_df.columns:
                    value = pivot_df.loc[idx, col]
                    ws.cell(row=current_row, column=col_idx, value=value)
                    data_cell = ws.cell(row=current_row, column=col_idx)
                    
                    # Apply cell formatting based on content
                    if idx == "Grand Total":
                        data_cell.font = Font(bold=True, size=9)
                        data_cell.fill = total_row_fill
                    else:
                        data_cell.font = Font(size=9)
                    
                    # Format numbers based on metric type
                    metric = col[1]  # Second level (metric)
                    if 'CVR' in metric:
                        data_cell.number_format = '0.00%'
                        # No conditional coloring per user request
                    elif 'AOV' in metric:
                        data_cell.number_format = '#,##0.00'
                    elif 'revenue' in metric.lower():
                        data_cell.number_format = '#,##0.00'
                    else:
                        data_cell.number_format = '#,##0'
                    
                    # Apply borders and alignment
                    data_cell.alignment = Alignment(horizontal='right', vertical='center')
                    data_cell.border = thin_border
                    
                    col_idx += 1
                
                current_row += 1
            
            # Auto-adjust column widths for better readability
            for col_idx in range(1, total_columns + 1):
                col_letter = get_column_letter(col_idx)
                max_length = 0
                for row_idx in range(start_row, current_row):
                    cell = ws.cell(row=row_idx, column=col_idx)
                    if cell.value:
                        try:
                            cell_length = len(str(cell.value))
                            if cell_length > max_length:
                                max_length = cell_length
                        except:
                            pass
                
                # Set width with padding, limit max width
                adjusted_width = min(max_length + 3, 25)
                ws.column_dimensions[col_letter].width = adjusted_width
            
            print(f"✅ Beautifully formatted sessions data written to {sheet_name} successfully!")
        else:
            print(f"❌ Sheet {sheet_name} not found in workbook")

    # Write each pivot table to its respective sheet
    write_sessions_pivot_to_sheet(master_sessions_pivot_no_filter, "Overall_IDG_Analysis")
    write_sessions_pivot_to_sheet(master_sessions_pivot_ea_only, "EA_IDG_Analysis")
    write_sessions_pivot_to_sheet(master_sessions_pivot_non_ea, "JumboAE_IDG_Analysis")

    # Save the workbook
    wb.save(output_file)
    print(f"\n✅ Successfully appended beautifully formatted sessions data to {output_file}")
    print("📊 Sessions analysis has been added to:")
    print("  • Overall_IDG_Analysis sheet")
    print("  • EA_IDG_Analysis sheet")
    print("  • JumboAE_IDG_Analysis sheet")
    
    print("\n🎨 Formatting Features Applied:")
    print("  • Professional color-coded headers and sections")
    print("  • Multi-level column headers with proper alignment")
    print("  • Highlighted Grand Total row")
    print("  • Color-coded CVR percentages (green/red)")
    print("  • Consistent borders and cell styling")
    print("  • Auto-adjusted column widths")
    print("  • Proper number formatting for different metrics")
    print("  • Clean visual separation between sections")

except Exception as e:
    print(f"❌ Error while exporting to Excel: {str(e)}")
    import traceback
    traceback.print_exc()  # Print full traceback for better debugging

print("\n" + "=" * 80)

In [ ]:
# Optional: Open the comprehensive Excel file automatically (Windows)
import os
import subprocess

# Update to use the correct comprehensive file name
output_file = 'IDG_Weekly_Analysis_Combined.xlsx'

try:
    # Check if the file exists
    if os.path.exists(output_file):
        print(f"🚀 Opening {output_file} in Excel...")
        # Use the default program to open the Excel file
        os.startfile(output_file)
        print("✅ Excel file opened successfully!")
        print(f"📊 Navigate through the sheets to explore all analyses:")
        print(f"   • Combined_IDG_Analysis for comprehensive view")
        print(f"   • Summary Dashboard for quick overview")
        print(f"   • Weekly Totals for growth tracking")
    else:
        print(f"❌ File {output_file} not found!")
except Exception as e:
    print(f"⚠️ Could not open Excel file automatically: {e}")
    print(f"Please manually open: {output_file}")

print("\n" + "=" * 70)
print("🎉 COMPLETE IDG WEEKLY ANALYSIS EXPORT FINISHED!")
print("=" * 70)
print("📊 ANALYSIS SUMMARY:")
print(f"• 📈 Overall Analysis: {len(all_idgs_global)} IDGs across {max_week_global} weeks")
print(f"• 🏢 EA Only Analysis: {len(all_idgs_ea)} IDGs across {max_week_ea} weeks")
print(f"• 🛍️ Jumbo.ae Analysis: {len(all_idgs_jumbo)} IDGs across {max_week_jumbo} weeks")
print(f"• 📅 Periods Analyzed: {', '.join(periods)}")
print(f"• 🗺️ Week Calculation: Based on {start_day} as first day of month")
print(f"• 📎 Export File: {output_file}")
print("\n🏆 All pivot tables created with comprehensive analysis!")
print("📊 Ready for business insights and decision making!")
print("=" * 70)